In [19]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Below are a few shorter problems. Please fill in the cells with your answer.

### Problem 1

Given the example below, we have a bidirectional GRU. What is the connection between output and hidden? Explain in detail where exactly in output you can find hidden, and why.

In [2]:
gru = nn.GRU(1, 1, bidirectional=True, batch_first=True)

# Use this same data for Problems 1 - 4
x = torch.rand(4, 5, 1)

# What is true about hidden and output? Where in output are the values in hidden? Be careful!
output, hidden = gru(x)

### Answer

The key is to realize that output has data like $(\overrightarrow{h}_t, \overleftarrow{h}_t)$ where $\overrightarrow{h}$ and $\overleftarrow{h}$ are respectively the forward and backward GRU's hidden states. Hidden, however, has the "last" hidden layer for each model. Specifically, for a sequence of length $T$, this is $(\overrightarrow{h}_{T}, \overleftarrow{h}_{1})$, becuase the forward model goes left to right while the back model goes right to left. I.e. you CANNOT take the last element of the output tensor state to get the same exact thing as the hidden tensor, you need to be careful.

### Problem 2

Consider the case when you have num_layers = 2 in a GRU as below. Describe what the conection now is between the hidden layer and the output layer. Specifically, what part of the 

In [3]:
gru = nn.GRU(1, 1, num_layers=2, batch_first=True)

# What is true about hidden and output? Where in output are the values in hidden? Be careful!
output, hidden = gru(x)

### Answer

Assume that the hidden layers are $\overrightarrow{h}^{1, 2}_t$. The thing to realize here is that output contains just the representation (across time, for each batch) of all the second (output) hidden layers. This, output is really just $(\overrightarrow{h}^{2}_1, \ldots, \overrightarrow{h}^{2}_T)$ assuming the batch is 1 and $T$ is the length of the data. On the other hand, hidden has in it $(\overrightarrow{h}^{1}_{T}, \overleftarrow{h}^{2}_{T})$, which is the final hidden representation of both the last (second) layer and the first layer. So, again, you cannot really connect a slice of the output with a slice of the returned hidden tensor.

### Problem 3

Given Problem 2, write code to get the representation across all time steps $T$ of the first layer. I.e., write code below to get $(\overrightarrow{h}^{1}_1, \ldots, \overrightarrow{h}^{1}_T)$. Do this for a GRU with two layers. Note that "output" does not have what you want - you need to be a little clever to get this.

In [4]:
gru = nn.GRU(1, 1, num_layers=2, batch_first=True)

# What is true about hidden and output? Where in output are the values in hidden? Be careful!
output, hidden = gru(x)

In [5]:
# Answer
# One easy way to do this is to do this is manually. Just have two GRUs and hae one's output feed into the other.
# Then, loow through the named parameters of the gru and insert them into one or the other of the two grus above.
from collections import OrderedDict

gru1 = nn.GRU(1, 1, num_layers=1, batch_first=True)
gru2 = nn.GRU(1, 1, num_layers=1, batch_first=True)

sd1 = gru1.state_dict()
sd2 = gru2.state_dict()
sd = gru.state_dict()

for name, param in sd.items():
    if name in sd1:
        sd1[name] = param
    else:
        name = name.split('_')
        name[-1] = 'l0'
        name = '_'.join(name)
        sd2[name] = param
        
gru1.load_state_dict(sd1)
gru2.load_state_dict(sd2)
        
# What is true about hidden and output? Where in output are the values in hidden? Be careful!
output1, hidden1 = gru1(x)

output2, hidden2 = gru2(output1)

assert(torch.all(torch.eq(output, output2)))
assert(torch.all(torch.eq(hidden, torch.vstack((hidden1, hidden2)))))

### Problem 4

In this problem we want to deal with sequences that are not the same length. Suppose we have 3 sequnces of data $a, b, c$, where the length of $a, b$ and $c$ are $2, 3$ and $4$ respectively. Assume you want to do a batch operation where the batch consists of $a, b$ and $c$ and you want to run these through the model. At the end, you'd like to get the final hidden state for each sentence. One way to do this is to pad all the sequences so they are length 4 and feed the 3 by 4 vector into the GRU.
- What is the problem with doing this? What is ineffcient about it?
- Investigate how to do this using the 4 imports below. You may not need all of these functions.
- I.e. create a batch of size 3 containing the 3 tensors.
- Pass this batch through the GRU.
- Get the outputs, and print the last hidden states, as if you were passing through a, b and c manually.

In [6]:
from torch.nn.utils.rnn import pack_sequence, pad_sequence, pack_padded_sequence, pad_packed_sequence

# Each tensor is in (length, values) format
a = torch.randn(2, 1)
b = torch.randn(3, 1)
c = torch.randn(4, 1)

la, lb, lc = 2, 3, 4

rnn = nn.RNN(1, 1, num_layers=1, batch_first=True)

In [7]:
# Answer:
# One easy way to do this is to do this is manually. Just have two GRUs and hae one's output feed into the other
# Then, low through the named parameters of the gru and insert them into one or the other of the two grus above
# The main this is that if you pad and pass through, you always run data the same length through the rnn (for each row of the batch)
# Imagine if one row has length 2 and one element of a batch has length 1000
# Now, all these small length examples will still need to be processed 1000 steps, and almost all elements of output corresponding to their rows will be "garbage"

seq = [a, b, c]

padded = pad_sequence(seq, batch_first=True)

packed1 = pack_padded_sequence(padded, lengths=[la, lb, lc], enforce_sorted=False, batch_first=True)
# pack_padded_sequence is older, the below is a newer command
packed2 = pack_sequence(seq, enforce_sorted=False)

output_packed1, hidden_packed1 = rnn(packed1)
output_packed2, hidden_packed2 = rnn(packed2)

output_padded1, output_lengths1 = pad_packed_sequence(output_packed1, batch_first=True, total_length=4)
output_padded2, output_lengths12= pad_packed_sequence(output_packed2, batch_first=True, total_length=4)

assert(torch.all(torch.eq(output_padded1, output_padded2)))

This example is like the previous one in HW 9, but now we want a more complicated model with attention.

In [8]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [9]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [10]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [11]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [12]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis fiance avec elle .', 'i am engaged to her .']


In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        # Make the encoder a GRU and also make it bidirectional.
        # Let it have 2 layers in the vertical direction.
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attention_projection = nn.Linear(self.hidden_size * 2, self.max_length)
        self.output_projection = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        # Make the GRU be unidirectional and also with 1 hidden layer
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # (1, 1, H)
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        # (1, 2 * H)
        # Concatenate yt and kt to get a vector (y_t, k_{t-1})
        embedded_hidden = torch.cat((embedded[0], hidden[0]), 1)
        
        # (1, MAX_LENGTH)
        # Project the above vector to get a vector mixing the elements of the above
        # This vector will be used to get attention scores with all the encoder embeddings
        # Here, the scores are scores = W_a[y_{t}, k_{t-1}] + b_a where W_a an b_a are in self.attention_projection
        # You can have other formats here, but the one above is enough for this problem
        attention_scores = self.attention_projection(embedded_hidden)
        
        # (1, MAX_LENGTH)
        # Get the attention weights from the scores
        attention_weights = nn.Softmax(dim=1)(attention_scores)
        
        # (1, 1, H)
        # Multiply the weights by the hidden states (h_1, h_2, .., h_{T_x}) of the encoder
        # This should be a vector of the above dimensions, so you'll need unsqueeze
        # One way to do this is using torch.bmm on these unsqueezed vectors
        # This will be the at vector that mixed the encoder's hidden representations; "c_{t}"" in lecture
        attention_context = torch.bmm(
            attention_weights.unsqueeze(0),
            encoder_outputs.unsqueeze(0)
        )

        # (1, 2*H)
        # Concatenate (yt, at) to get a vector that we will use to predict the output
        output = torch.cat((embedded[0], attention_context[0]), 1)
        
        # (1, 1, H)
        # Project the above vector into a new vector we'll use to predict with
        output = self.output_projection(output).unsqueeze(0)

        # (1, H)
        output = F.relu(output)
        
        # (1, H) and (1, H)
        # Pass the output and hidden through the GRU. Note that we apply attention before we pass into the GRU
        # This is a bit of the reverse from lecture
        output, hidden = self.gru(output, hidden)

        # (1, V)
        # Either apply log_softmax to output or leave it alone
        # This will have you use the NLLLoss or the CrossEntropyLoss
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attention_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [15]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [16]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    target_length_used = 0

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        
        target_length_used = target_length
        
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        
        for di in range(target_length):
            
            target_length_used += 1
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length_used

In [17]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [18]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, it / n_iters),
                                         it, it / n_iters * 100, print_loss_avg))

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            showPlot(plot_losses)

In [ ]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttentionDecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder, decoder, 75000, print_every=5000)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            
            decoded_words.append(output_lang.index2word[topi.item()])
            
            if topi.item() == EOS_token:
                break                

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def evaluateRandomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        pair = random.choice(pairs)
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
        # Leave out the <EOS> symbol
        output_words, _ = evaluate(encoder, decoder, pair[0])
        
        if 'EOS' == output_words[-1]:
            output_words.pop()
                                        
        output_sentence = ' '.join(output_words)
        score = 100.0 * sentence_bleu([pair[1].split(' ')], output_words, weights=[0.5, 0.5])
        bleu_scores.append(score)
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    print('The mean BLEU score is: ', np.mean(bleu_scores))

In [ ]:
evaluateRandomly(encoder, decoder)